## Quiz 2

income_data.csv 파일을 이용하여  
Feature 특성에 맞게 데이터전처리하여  
Income 을 예측하는 LinearSVC 를 학습하여 정확도를 출력하시오.  


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.appName('my app').master('local').getOrCreate()
sc = spark.sparkContext

22/10/20 08:14:27 WARN Utils: Your hostname, cherry resolves to a loopback address: 127.0.1.1; using 192.168.0.7 instead (on interface enp6s0)
22/10/20 08:14:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 08:14:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 08:14:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [39]:
from pyspark.sql.types import *
import pyspark.sql.functions as fn
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.ml.classification import LinearSVC

In [ ]:
Columns  
  
age: Age of the person  
workclass: Categorical variable indicating the type of work i.e. Govt or Private etc,  
fnlwgt: final weight  
education: Education  
educational-num: Education as Integer  
marital-status: Marital Status  
occupation: Occupation  
relationship: Relationship Status  
race: Race of the Person  
gender: Gender of the person  
capital-gain: capital Gain  
capital-loss: Capital loss  

In [6]:
data.show(3)

+---+-----------------+------+----------+---------------+-------------------+------------------+--------------+------+------+------------+------------+--------------+--------------+------+
|age|        workclass|fnlwgt| education|educational-num|     marital-status|        occupation|  relationship|  race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+-----------------+------+----------+---------------+-------------------+------------------+--------------+------+------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516| Bachelors|             13|      Never-married|      Adm-clerical| Not-in-family| White|  Male|        2174|           0|            40| United-States| <=50K|
| 50| Self-emp-not-inc| 83311| Bachelors|             13| Married-civ-spouse|   Exec-managerial|       Husband| White|  Male|           0|           0|            13| United-States| <=50K|
| 38|          Private|215646|   HS-grad|              

In [49]:
schema = StructType([
    StructField('age', FloatType(), True),
    StructField('workclass', StringType(), True),
    StructField('fnlwgt', FloatType(), True),
    StructField('education', StringType(), True),
    StructField('educational-num', FloatType(), True),
    StructField('marital-status', StringType(), True),
    StructField('occupation', StringType(), True),
    StructField('relationship', StringType(), True),
    StructField('race', StringType(), True),
    StructField('gender', StringType(), True),
    StructField('capital-gain', FloatType(), True),
    StructField('capital-loss', FloatType(), True),
    StructField('hours-per-week', FloatType(), True),
    StructField('native-country', StringType(), True),
    StructField('income', StringType(), True),
])

data = spark.read.format('csv').option('header', 'true').schema(schema).load('./input_week08/income_data.csv')
data.printSchema()

root
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: float (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: float (nullable = true)
 |-- capital-loss: float (nullable = true)
 |-- hours-per-week: float (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [51]:
data_col = data.columns

In [9]:
data.summary().show()

+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------+
|summary|               age|   workclass|            fnlwgt|    education|  educational-num|marital-status|       occupation|relationship|               race| gender|      capital-gain|    capital-loss|    hours-per-week|native-country|income|
+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------+
|  count|             32561|       32561|             32561|        32561|            32561|         32561|            32561|       32561|              32561|  32561|             32561|           32561|             32561|         32561| 32561|
|   mean| 38.58164675532

In [52]:
data.rdd.map(
    lambda row: 1 if sum([c=='?' for c in row]) > 0 else 0
).reduce(
    lambda x, y: x+y
)

0

In [53]:
for c in data_col:
    data = data.withColumn(c, fn.when(fn.col(c)=='?', None).otherwise(fn.col(c)))

data.show(5)

+----+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| age|        workclass|  fnlwgt| education|educational-num|     marital-status|        occupation|  relationship|  race| gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+----+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|39.0|        State-gov| 77516.0| Bachelors|           13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
|50.0| Self-emp-not-inc| 83311.0| Bachelors|           13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
|38.0|          Private|215646.0|  

In [54]:
for c in data_col:
    data = data.withColumn(c, fn.when(fn.col(c)==0, None).otherwise(fn.col(c)))

data.show(5)

+----+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| age|        workclass|  fnlwgt| education|educational-num|     marital-status|        occupation|  relationship|  race| gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+----+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|39.0|        State-gov| 77516.0| Bachelors|           13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|        null|          40.0| United-States| <=50K|
|50.0| Self-emp-not-inc| 83311.0| Bachelors|           13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|        null|        null|          13.0| United-States| <=50K|
|38.0|          Private|215646.0|  

In [55]:
data.select(*[
    (fn.count(c) / fn.count('*')).alias(c + '_missing') for c in data_col
]).show()

+-----------+-----------------+--------------+-----------------+-----------------------+----------------------+------------------+--------------------+------------+--------------+--------------------+--------------------+----------------------+----------------------+--------------+
|age_missing|workclass_missing|fnlwgt_missing|education_missing|educational-num_missing|marital-status_missing|occupation_missing|relationship_missing|race_missing|gender_missing|capital-gain_missing|capital-loss_missing|hours-per-week_missing|native-country_missing|income_missing|
+-----------+-----------------+--------------+-----------------+-----------------------+----------------------+------------------+--------------------+------------+--------------+--------------------+--------------------+----------------------+----------------------+--------------+
|        1.0|              1.0|           1.0|              1.0|                    1.0|                   1.0|               1.0|                 1.0|

In [56]:
data_col.remove('capital-gain')
data_col.remove('capital-loss')

In [57]:
data = data.select(data_col)
data.show(3)

+----+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+------+--------------+--------------+------+
| age|        workclass|  fnlwgt| education|educational-num|     marital-status|        occupation|  relationship|  race|gender|hours-per-week|native-country|income|
+----+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+------+--------------+--------------+------+
|39.0|        State-gov| 77516.0| Bachelors|           13.0|      Never-married|      Adm-clerical| Not-in-family| White|  Male|          40.0| United-States| <=50K|
|50.0| Self-emp-not-inc| 83311.0| Bachelors|           13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|  Male|          13.0| United-States| <=50K|
|38.0|          Private|215646.0|   HS-grad|            9.0|           Divorced| Handlers-cleaners| Not-in-family| White|  Male|          40.0| United-States| <=50K|
+---

In [80]:
string_col = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
string_col_idx = ['workclass_idx', 'education_idx', 'marital-status_idx', 'occupation_idx', 'relationship_idx', 'race_idx', 'gender_idx', 'native-country_idx', 'income_idx']

In [84]:
indexer = StringIndexer(inputCols=string_col, outputCols=string_col_idx)
model = indexer.fit(data)
data_convert = model.transform(data)
data_convert = data_convert.select(['age', 'workclass_idx', 'fnlwgt', 'education_idx', 'educational-num', 'marital-status_idx', 'occupation_idx', 'relationship_idx', 'race_idx', 'gender_idx', 'hours-per-week', 'native-country_idx', 'income_idx'])

In [82]:
data_convert.show(3)

+----+-------------+--------+-------------+---------------+------------------+--------------+----------------+--------+----------+--------------+------------------+----------+
| age|workclass_idx|  fnlwgt|education_idx|educational-num|marital-status_idx|occupation_idx|relationship_idx|race_idx|gender_idx|hours-per-week|native-country_idx|income_idx|
+----+-------------+--------+-------------+---------------+------------------+--------------+----------------+--------+----------+--------------+------------------+----------+
|39.0|          4.0| 77516.0|          2.0|           13.0|               1.0|           3.0|             1.0|     0.0|       0.0|          40.0|               0.0|       0.0|
|50.0|          1.0| 83311.0|          2.0|           13.0|               0.0|           2.0|             0.0|     0.0|       0.0|          13.0|               0.0|       0.0|
|38.0|          0.0|215646.0|          0.0|            9.0|               2.0|           9.0|             1.0|     0.0| 

In [89]:
cols = data_convert.columns
cols.remove('income_idx')

In [90]:
assembler = VectorAssembler(inputCols=cols, outputCol='features')
data_convert = assembler.transform(data_convert)
data_convert.select('features').show(5, truncate=False)

+---------------------------------------------------------+
|features                                                 |
+---------------------------------------------------------+
|[39.0,4.0,77516.0,2.0,13.0,1.0,3.0,1.0,0.0,0.0,40.0,0.0] |
|[50.0,1.0,83311.0,2.0,13.0,0.0,2.0,0.0,0.0,0.0,13.0,0.0] |
|[38.0,0.0,215646.0,0.0,9.0,2.0,9.0,1.0,0.0,0.0,40.0,0.0] |
|[53.0,0.0,234721.0,5.0,7.0,0.0,9.0,0.0,1.0,0.0,40.0,0.0] |
|[28.0,0.0,338409.0,2.0,13.0,0.0,0.0,4.0,1.0,1.0,40.0,9.0]|
+---------------------------------------------------------+
only showing top 5 rows



In [91]:
standardscaler = StandardScaler(
    inputCol='features', outputCol='scaled_features', withStd=True, withMean=False
)

data_convert = standardscaler.fit(data_convert).transform(data_convert)
data_convert.select('features', 'scaled_features').show(5)

+--------------------+--------------------+
|            features|     scaled_features|
+--------------------+--------------------+
|[39.0,4.0,77516.0...|[2.85914686699289...|
|[50.0,1.0,83311.0...|[3.66557290640114...|
|[38.0,0.0,215646....|[2.78583540886487...|
|[53.0,0.0,234721....|[3.88550728078521...|
|[28.0,0.0,338409....|[2.05272082758464...|
+--------------------+--------------------+
only showing top 5 rows



In [92]:
train, test = data_convert.randomSplit([0.8, 0.2])

print(train.count())
print(test.count())

26123
6438


In [94]:
lr = LinearSVC(
    labelCol='income_idx',
    featuresCol='scaled_features',
    maxIter=10
)

model = lr.fit(train)
predict_train = model.transform(train)
predict_test = model.transform(test)

22/10/20 09:27:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/20 09:27:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [95]:
df_output = predict_test.select('income_idx', 'prediction')
df_filter = df_output.filter(df_output['income_idx'] == df_output['prediction'])

In [96]:
df_filter.count() / df_output.count()

0.8191985088536813